## File descriptions
* *sales_train.csv* - the training set. Daily historical data from January 2013 to October 2015.
* *test.csv* - the test set. You need to forecast the sales for these shops and products for November 2015.
* *sample_submission.csv* - a sample submission file in the correct format.
* *items.csv* - supplemental information about the items/products.
* *item_categories.csv*  - supplemental information about the items categories.
* *shops.csv* - supplemental information about the shops.


## Data fields
* *ID* - an Id that represents a (Shop, Item) tuple within the test set
* *shop_id* - unique identifier of a shop
* *item_id* - unique identifier of a product
* *item_category_id* - unique identifier of item category
* *item_cnt_day* - number of products sold. You are predicting a monthly amount of this measure
* *item_price* - current price of an item
* *date* - date in format dd/mm/yyyy
* *date_block_num* - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* *item_name* - name of item
* *shop_name* - name of shop
* *item_category_name* - name of item category


In [16]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta

#pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)


In [17]:
df_shops = pd.read_csv("shops.csv")
df_shops.describe()

,shop_id
count,60.00000
mean,29.50000
std,17.46425
min,0.00000
25%,14.75000
50%,29.50000
75%,44.25000
max,59.00000


In [18]:
df_categories = pd.read_csv("item_categories.csv")
df_categories.describe()

,item_category_id
count,84.00000
mean,41.50000
std,24.39262
min,0.00000
25%,20.75000
50%,41.50000
75%,62.25000
max,83.00000


In [19]:
df_items = pd.read_csv("items.csv")
df_items.describe()

,item_id,item_category_id
count,22170.00000,22170.00000
mean,11084.50000,46.29075
std,6400.07207,15.94149
min,0.00000,0.00000
25%,5542.25000,37.00000
50%,11084.50000,40.00000
75%,16626.75000,58.00000
max,22169.00000,83.00000


In [20]:
df_sales = pd.read_csv("sales_train.csv")
df_sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2935849.00000,2935849.00000,2935849.00000,2935849.00000,2935849.00000
mean,14.56991,33.00173,10197.22706,890.85323,1.24264
std,9.42299,16.22697,6324.29735,1729.79963,2.61883
min,0.00000,0.00000,0.00000,-1.00000,-22.00000
25%,7.00000,22.00000,4476.00000,249.00000,1.00000
50%,14.00000,31.00000,9343.00000,399.00000,1.00000
75%,23.00000,47.00000,15684.00000,999.00000,1.00000
max,33.00000,59.00000,22169.00000,307980.00000,2169.00000


In [22]:
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00000,1.00000
1,03.01.2013,0,25,2552,899.00000,1.00000
2,05.01.2013,0,25,2552,899.00000,-1.00000
3,06.01.2013,0,25,2554,1709.05000,1.00000
4,15.01.2013,0,25,2555,1099.00000,1.00000


In [233]:
#df_sales.loc[(df_sales['date']=='13.01.2013') & df_sales['shop_id'].isin([0,1,57,58])]
#df_sales.loc[df_sales['shop_id'].isin([39, 40])]
#df_sales.loc[df_sales['shop_id'] == 40]
#df_sales.loc[~df_sales['item_id'].isin(df_items['item_id'].values)]
df_sales.loc[df_sales['item_cnt_day'] < 0]['shop_id'].value_counts()
#df_sales.loc[(df_sales['item_id'] == 11373) & (df_sales['shop_id'] == 5)]
#df_sales.loc[(df_sales['item_id'] == 11373)]['shop_id'].value_counts()

31    451
12    430
54    356
25    316
57    314
6     261
42    251
28    216
43    211
19    211
56    195
47    193
58    173
44    165
53    163
38    162
27    157
37    154
29    151
52    143
16    139
59    137
30    135
26    134
18    132
50    130
15    130
35    128
21    125
4     124
14    120
3     119
45    103
24    103
46    102
7      99
22     97
48     89
5      85
41     82
2      80
17     80
49     42
39     37
32     34
34     25
33     22
23     17
40     13
9       8
8       6
36      3
55      2
20      1
Name: shop_id, dtype: int64

In [50]:
#df_sales.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].sum()
df_mean = df_sales.groupby(['shop_id','item_id'])['item_cnt_day'].mean()
df_mean.describe()

count   424124.00000
mean         1.06407
std          1.14536
min         -1.75000
25%          1.00000
50%          1.00000
75%          1.00000
max        666.66667
Name: item_cnt_day, dtype: float64

In [27]:
df_shops.head(60)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [28]:
df_test = pd.read_csv("test.csv")
df_test.describe()

,ID,shop_id,item_id
count,214200.00000,214200.00000,214200.00000
mean,107099.50000,31.64286,11019.39863
std,61834.35817,17.56193,6252.64459
min,0.00000,2.00000,30.00000
25%,53549.75000,16.00000,5381.50000
50%,107099.50000,34.50000,11203.00000
75%,160649.25000,47.00000,16071.50000
max,214199.00000,59.00000,22167.00000


In [47]:
valid_shops = np.sort(df_test['shop_id'].unique())
df_shops.loc[~df_shops['shop_id'].isin(valid_shops)] 

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9
11,Жуковский ул. Чкалова 39м²,11
13,"Казань ТЦ ""Бехетле""",13
17,"Красноярск ТЦ ""Взлетка Плаза""",17
20,"Москва ""Распродажа""",20
23,"Москва ТК ""Буденовский"" (пав.А2)",23
27,"Москва ТЦ ""МЕГА Белая Дача II""",27


In [129]:
# get valid shop id
def valid_shop_id(id):
    if id == 0:
        return 57
    if id == 1:
        return 58
    if id == 23:
        return 24
    if id == 11:
        return 10
    if id == 40:
        return 39
    return id

def shop_type(shop):
    #types = ['ТК', 'ТЦ', 'ТРК', 'ТРЦ']
    p = re.compile(r"(Т[РКЦ]+)")
    r = p.search(shop)
    if r is not None: return r.group(1) 
    if 'нлайн' in shop: return 'Онлайн'
    if 'нтернет' in shop: return 'Онлайн'
    return 'Магазин'

# get city
def shop_city(shop):
    p = re.compile(r"^([а-яА-Я]*)")
    r = p.search(shop)
    if r is not None: return r.group(1) 
    return 'Unknown'

#df_shops['valid_id'] = df_shops['shop_id'] 

In [133]:
#print(shop_type('Выездная Торговля'))
print(shop_city('Москва ТЦ "МЕГА Белая Дача II"'))
print(shop_city('!Якутск Орджоникидзе, 56 фран'))

Москва



In [136]:
df_shops.drop([0,1], inplace=True)

In [147]:

df_shops['city'] = df_shops['shop_name'].apply(lambda x: shop_city(x))
df_shops['type'] = df_shops['shop_name'].apply(lambda x: shop_type(x))
df_shops['mega'] = df_shops['shop_name'].str.contains('мега', case=False).astype(int)

In [148]:
df_shops

,shop_name,shop_id,city,type,mega
2,"Адыгея ТЦ ""Мега""",2,Адыгея,ТЦ,1
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,ТРК,0
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,ТЦ,0
5,"Вологда ТРЦ ""Мармелад""",5,Вологда,ТРЦ,0
6,"Воронеж (Плехановская, 13)",6,Воронеж,Магазин,0
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж,ТРЦ,0
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж,ТРЦ,0
9,Выездная Торговля,9,Выездная,Магазин,0
10,Жуковский ул. Чкалова 39м?,10,Жуковский,Магазин,0
11,Жуковский ул. Чкалова 39м²,11,Жуковский,Магазин,0


In [181]:
df_categories['category'] = df_categories['item_category_name'].str.split('[-(]', n=0).str[0].str.strip()
df_categories['digital'] = df_categories['item_category_name'].str.contains('цифра', case=False).astype(int)

In [183]:
df_categories.head(10)

,item_category_name,item_category_id,category,digital
0,PC - Гарнитуры/Наушники,0,PC,0
1,Аксессуары - PS2,1,Аксессуары,0
2,Аксессуары - PS3,2,Аксессуары,0
3,Аксессуары - PS4,3,Аксессуары,0
4,Аксессуары - PSP,4,Аксессуары,0
5,Аксессуары - PSVita,5,Аксессуары,0
6,Аксессуары - XBOX 360,6,Аксессуары,0
7,Аксессуары - XBOX ONE,7,Аксессуары,0
8,Билеты (Цифра),8,Билеты,1
9,Доставка товара,9,Доставка товара,0


In [235]:
def get_category(item_id):
    category_id = df_items.loc[df_items['item_id'] == item_id]['item_category_id'].values[0]
    return df_categories.loc[df_categories['item_category_id'] == category_id]['category'].values[0]

def get_category_digitality(item_id):
    category_id = df_items.loc[df_items['item_id'] == item_id]['item_category_id'].values[0]
    return df_categories.loc[df_categories['item_category_id'] == category_id]['digital'].values[0]

def get_shop_type(shop_id):
    return df_shops.loc[df_shops['shop_id'] == shop_id]['type'].values[0]

def get_shop_megality(shop_id):
    return df_shops.loc[df_shops['shop_id'] == shop_id]['mega'].values[0]


In [239]:
#get_category_digitability(8)
#get_shop_megality(27)
df_sales['shop_id'] = df_sales['shop_id'].apply(lambda x: valid_shop_id(x))

In [ ]:
df_sales['shop_type'] = df_sales['shop_id'].apply(lambda x: get_shop_type(x))

In [ ]:
df_sales['shop_mega'] = df_sales['shop_id'].apply(lambda x: get_shop_megality(x))

In [ ]:
df_sales['category'] = df_sales['item_id'].apply(lambda x: get_category(x))

In [ ]:
df_sales['digital'] = df_sales['item_id'].apply(lambda x: get_category_digitality(x))
df_sales.head(10)